In [4]:
import json
import re
with open("../dataset/smiles_test.json") as f:
    dataset = json.load(f)
    dataset = dataset[:1000]

with open("result_brics.json", "r") as f:
    results = json.load(f)

In [56]:
def remove_pre(s):
    pattern = re.compile(r'^\d+\.')
    return pattern.sub('', s).lstrip().strip()

def remove_pattern(text):
    # 使用正则表达式匹配并去除 "[数字*]" 模式
    result = re.sub(r'\[\d+\*\]', '', text)
    return result.replace("'", '"')

In [67]:
from tqdm import tqdm

origin = []
for item in tqdm(dataset):
    l = item['output'].split('\n')[1:]
    res = [remove_pattern(remove_pre(i)) for i in l]
    res = sorted(res)
    origin.append(res)

print(origin)
    

100%|██████████| 1000/1000 [00:00<00:00, 73952.75it/s]

[['N1N=C(C)[C@@]2(C(=O)C=CC[C@H]2)C1=O', 'c1ccc(C)cc1'], ['C', 'C()=O', 'N', 'O', 'c1ccc()cc1', 'c1cccc(Cl)c1C', 'c1ccccc1', 'c1nc()c(C)o1'], ['C', 'C()=O', 'C(=O)CCCCC', 'C(C)C', 'CC', 'N()', 'O', 'c1ccc()cc1', 'c1cccc()c1', 'c1nnc(C)o1'], ['CC', 'CC', 'N(C)C', 'O', 'OC', 'OCO/N=C1/[C@H](C)C[C@]()(C)[C@H]()[C@@H](C)C(=O)[C@@H](C)C(=O)O[C@H]()[C@@](C)(O)[C@H](O)[C@H]1C', 'OCO/N=C1/[C@H](C)C[C@]()(C)[C@H]()[C@@H](C)C(=O)[C@@H](C)C(=O)O[C@H]()[C@@](C)(O)[C@H](O)[C@H]1C', '[C@@H]1O[C@H](C)C[C@H]()[C@H]1O'], ['C/N=c1\\c(O)c(O)\\c1=N/C(C)(C)C', 'CC', 'c1ccc(C#N)cc1'], ['C(=O)C', 'C(N)=O', 'N', 'c1ccc2nccnc2c1', 'c1cccc()c1', 'c1cccc(C)n1', 'n1cc()c()n1'], ['C', 'C()=O', 'C1CC1', 'CC', 'N', 'N()', 'OC', '[C@H]1CCNC[C@@H]1', 'c1cc()c(Cl)c(Cl)c1', 'c1ccn(C)c(=O)c1'], ['C(=O)C', 'CC', 'N1CCCCC1', 'S', 'c1cccc(Cl)c1', 'c1nnc()o1'], ['C', 'C', 'C()=O', 'C(=O)C', 'C(=O)C', 'N', 'N()C', 'NC', 'O', 'c1ccc()cc1', 'c1ccc(Cl)c()c1Cl', 'c1cccn2c(Br)c(C)nc12'], ['C', 'C()=O', 'C1CC1', 'N1CCN=C1S', 'c1ccc

In [68]:
output = []
for item in tqdm(results):
    l = item[0].split('\n')
    pattern = re.compile(r'^\d+\..*')
    filtered_list = [remove_pattern(remove_pre(s)) for s in l if pattern.match(s)]
    filtered_list = sorted(filtered_list)
    output.append(filtered_list)
print(output)

100%|██████████| 1000/1000 [00:00<00:00, 79867.16it/s]

[['N1N=C(C)C(=O)[C@@]12C(=O)C=CC[C', '[C@]12C(=O)C=CC[C@H]2c1ccc(C)cc1', 'c1ccc(C)cc1'], ['C()=O', 'N', 'O', 'c1cccc(Cl)c1C', 'c1ccccc1', 'c1nc()c(C)c1'], ['', 'C()=O', 'C(C)C', 'N', 'O', 'c1cccc()c1'], ['O[3'], ['C/N=c1\\c(O)c(O)\\c1=N/C(C)(C)C', 'CC', 'c1ccc(C#N)cc1'], ['C(=O)C', 'C(N)=O', 'N', 'c1cccc()c1', 'c1cccc(C)n1', 'n1cc'], ['C', 'C()=O', 'N', 'N1CC[C@H]()[C@@H]1', '[4'], ['C(=O)C', 'N1CCCCC(C)N1', 'S', 'c1cccc(Cl)c1', 'c1nnc()o1'], ['C', 'C(=O)C', 'N', 'O', 'c1ccc(Cl)c()c1Cl'], ['C()=O', 'C1CC1', 'N1CCN=C1S', 'c1ccc(Cl)cc1'], ['C(=O)C', 'N', 'OC', 'c1ccc()cc1', 'n1cnc2scc()c2c1=O'], ['C(=O)C(=N/Nc1ccccc1)N1CCCc2ccccc21', 'CC', 'O', 'OC'], ['C(=O)N/N=C1\\CCCc2c1c1ccc([nH]2)c()c1', 'C(C)C', 'c1ccncc1'], ['c1c2c(c()ccccc2[nH]1)NCC1CCClc2c1c1ccccc21'], ['C(F)(F)F', 'C1CCCCC1', 'N()', 'c1cc()nc()n1', 'c1ccco1'], ['c1cc2cc(O)c(O)cc2c(C)c1C', 'c1ccc(F)cc1'], ['C()=O', 'CC', 'N', 'c1ccc(/C(Br)=C2\\C[C@H]3CC[C@@H](C2)N3C[8*'], ['c1cc2ccccc2n1C', 'c1cc2ccccc2n1C', 'c1nsc()n1'], ['C', 

In [70]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
total_scores = {'rouge1': 0, 'rougeL': 0}
num_pairs = len(origin)

for i,j in zip(origin, output):
    scores = scorer.score(" ".join(i), " ".join(j))
    # print(scores)
    for key in total_scores:
        total_scores[key] += scores[key].fmeasure

average_scores = {key: value / num_pairs for key, value in total_scores.items()}

print(average_scores)

{'rouge1': 0.7328690184286074, 'rougeL': 0.6889101716377377}
